In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import glob
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
CLASSES_PATHS = glob.glob('train_small/*')
NUM_CLASSES = len(CLASSES_PATHS)


print('Number of classes : ', NUM_CLASSES)
print('------------------------------------------')

for path in CLASSES_PATHS:
    print(path.split('/')[-1], '----->', len(glob.glob(path + '/*')))

Number of classes :  137
------------------------------------------
Terebratulina_sp_ -----> 15
Hermit_crab -----> 40
Pecten_sulcicostatus -----> 10
Polychaete_worms -----> 80
Armina_sp_ -----> 16
Fusinus_africanae -----> 13
Ophiothrix_fragilis -----> 29
Vitjazmaia_latidactyla -----> 10
Holothuroidea -----> 13
Astropecten_irregularis_pontoporeus -----> 43
Seafan -----> 34
Nassarius_vinctus -----> 45
Dipsacaster_sladeni_capensis -----> 16
Exodromidia_spinosissima -----> 10
Pseudodromia_spp_ -----> 16
Anthoptilum_grandiflorum -----> 22
Scleractinia -----> 83
Marginella_musica -----> 30
Salpa_spp_ -----> 19
Cephalodiscus_gilchristi -----> 10
Bolocera_kerguelensis -----> 28
Ceramaster_patagonicus_euryplax -----> 12
Hippasteria_phrygiana -----> 10
Ophiura_costata_costata -----> 37
Ophiura_trimeni -----> 12
Halcurias_capensis -----> 11
Nassarius speciosus -----> 13
Limopsis_chuni -----> 14
Aphelodoris_sp_ -----> 12
Nudibranchia -----> 10
Isididae -----> 17
Amalda_bullioides -----> 25
Pseudar

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = (256, 256)
EPOCHS = 50

train_ds = image_dataset_from_directory('train_small',
                                        subset='training',
                                        validation_split=0.05,
                                        seed=43,
                                        image_size=IMAGE_SIZE,
                                        batch_size=BATCH_SIZE)

val_ds = image_dataset_from_directory('train_small',
                                      subset='validation',
                                      validation_split=0.05,
                                      seed=43,
                                      image_size=IMAGE_SIZE,
                                      batch_size=BATCH_SIZE)

Found 3111 files belonging to 137 classes.
Using 2956 files for training.
Found 3111 files belonging to 137 classes.
Using 155 files for validation.


In [ ]:
resnet_url = 'https://tfhub.dev/google/experts/bit/r50x1/in21k/invertebrate/1'
resnet50 = hub.KerasLayer(resnet_url)


In [ ]:
inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
rescaled = tf.keras.layers.Rescaling(1/255)(inputs)
encoded = resnet50(rescaled)
outputs = Dense(NUM_CLASSES, activation='softmax')(encoded)

model = Model(inputs=[inputs], outputs=[outputs])

In [ ]:
def scheduler(epoch):
    lr = 0.001
    div = 1
    if epoch+1 % 10 == 0:
        div = div/epoch
    return lr/div


calls = [tf.keras.callbacks.LearningRateScheduler(scheduler)]

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=calls)

Epoch 1/50
24/24 [==============================] - 22s 720ms/step - loss: 1.5845 - accuracy: 0.6005 - val_loss: 1.0282 - val_accuracy: 0.6774
Epoch 2/50
24/24 [==============================] - 18s 684ms/step - loss: 0.4561 - accuracy: 0.8654 - val_loss: 0.7079 - val_accuracy: 0.8258
Epoch 3/50
24/24 [==============================] - 18s 682ms/step - loss: 0.2117 - accuracy: 0.9432 - val_loss: 0.4985 - val_accuracy: 0.8516
Epoch 4/50
24/24 [==============================] - 18s 685ms/step - loss: 0.1190 - accuracy: 0.9696 - val_loss: 0.3978 - val_accuracy: 0.8839
Epoch 5/50
24/24 [==============================] - 18s 687ms/step - loss: 0.0737 - accuracy: 0.9882 - val_loss: 0.3211 - val_accuracy: 0.9290
Epoch 6/50
24/24 [==============================] - 18s 685ms/step - loss: 0.0538 - accuracy: 0.9912 - val_loss: 0.3850 - val_accuracy: 0.9161
Epoch 7/50
24/24 [==============================] - 18s 687ms/step - loss: 0.0424 - accuracy: 0.9932 - val_loss: 0.3019 - val_accuracy: 0.9161

In [ ]:
# Save model

save_dir = '/content/drive/MyDrive/sub19'
tf.saved_model.save(model, save_dir)

# Load model

# model = hub.KerasLayer(save_dir, trainable=True)


In [ ]:
images_names = []
images = []
test_paths = glob.glob('test_small/*')

for path in test_paths:
    img = tf.keras.utils.load_img(path, target_size=IMAGE_SIZE)
    img = tf.keras.preprocessing.image.img_to_array(img)
    images.append(img)
    images_names.append(path.split('/')[-1])

images = np.array(images)


In [ ]:
columns = ['FILE'] + train_ds.class_names
submission = pd.DataFrame(columns=columns)
submission['FILE'] = images_names

In [ ]:
predicted = model.predict(images)
submission[train_ds.class_names] = predicted


In [ ]:
simple_sub = pd.read_csv('/content/SampleSubmission_.csv')
reindex_c = simple_sub.columns
submission = submission[reindex_c]

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head()

,FILE,Pteraster_capensis,Porifera,Astropecten_irregularis_pontoporeus,Terebratulina_sp_,Neolithodes_asperrimus,Munida_benguela,Cypraeovula_iutsui,Kaloplocamus_ramosus,Actinoscyphia_plebeia,Echinus_gilchristi,Synallactes_viridilimus,Eleutherobia_variable,Neopilumnoplax_heterochir,Pleurobranchaea_bubala,Pagurus_cuanensis,Parapagurus_bouvieri,Inachidae,Pseudarchaster_tessellatus,Pasiphaea_sp._1,Suberites_dandelenae,Comitas_saldanhae,Poraniopsis_echinaster,Pterygosquilla_capensis,Holothuroidea,Exodromidia_spinosa,Aristeus_varidens,Philinopsis_capensis,Vitjazmaia_latidactyla,Perissasterias_polyacantha,Ascidiacea,Toraster_tuberculatus,Nudibranchia,Mediaster_bairdi_capensis,Limopsis_chuni,Comitas_stolida,Psilaster_acuminatus,Lithodes_ferox,Hermit_crab,Solenocera_africana,...,Ophiura_costata_costata,Pelagia_noctiluca,Sclerasterias_spp,Stylasteridae,Fusivoluta_pyrrhostoma,Actinoptilum_molle,Pseudarchaster_brachyactis,Nassarius speciosus,Chondraster_elattosis,Ophiothrix_aristulata,Aphelodoris_sp_,Fusinus_africanae,Scaphander_punctostriatus,Polychaete_tubes_(only),Polychaete_worms,Polyechinus_agulhensis,Chrysaora_spp,Projasus_parkeri,Lamellaria_Coriocella_spp,Hippasteria_phrygiana,Mursia_cristiata,Prawns,Nassarius_vinctus,Marthasterias_africana,Goneplax_clevai,Alcyonacea,Athleta_lutosa,Pseudodromia_spp_,Rochinia_hertwigi,Africolaria_rutila,Neptuneopsis_gilchristi,Pseudodromia_rotunda,Pecten_sulcicostatus,Plesionika_martia,Rossella_antarctica,Philine_aperta,Mycale_anisochela,Henricia_abyssalis,Triviella_spp_,Cheilostomatida
0,3EDA42G.jpeg,9.9348e-09,2.10595e-07,6.34133e-09,2.98159e-08,3.12871e-09,1.95343e-07,2.94228e-11,1.02313e-07,1.54003e-07,1.47246e-09,3.05354e-07,7.40592e-06,1.37594e-10,1.18963e-06,2.77622e-09,6.55752e-09,1.09701e-10,2.73671e-13,1.04492e-11,8.11178e-13,3.20341e-10,8.52668e-07,1.02101e-09,2.73167e-08,2.97028e-11,2.39622e-10,0.000146908,1.36518e-11,0.000137537,2.6604e-10,8.19688e-08,2.43095e-08,4.63883e-10,3.15072e-09,2.41585e-11,3.26403e-16,5.46833e-14,7.4401e-05,1.52339e-11,...,1.26381e-11,2.78199e-07,1.28681e-10,8.59906e-10,3.9394e-07,2.10762e-07,2.03597e-13,5.64244e-09,7.1068e-10,1.44318e-13,1.15049e-08,5.76817e-08,4.71611e-10,8.18628e-08,1.69459e-06,6.21255e-12,1.1794e-08,1.22138e-12,3.25366e-08,2.33671e-10,5.91454e-09,8.49062e-09,1.64149e-06,8.93073e-11,4.05126e-13,1.88335e-08,3.71049e-11,2.39149e-11,4.28602e-14,1.87635e-10,1.11009e-11,1.49939e-09,7.45523e-10,1.09519e-10,5.66498e-09,2.82662e-10,9.38931e-09,1.25378e-10,8.55387e-12,2.16947e-09
1,SXM1R5N.jpeg,0.000249814,4.13774e-07,3.29731e-10,2.33521e-11,1.00182e-12,3.67936e-15,1.16211e-12,6.67022e-13,8.70103e-07,0.000774714,1.75191e-12,1.06286e-09,7.719e-14,4.82377e-08,3.20581e-12,6.05155e-14,5.0314e-12,1.8679e-10,6.61994e-12,1.14689e-11,5.41057e-13,1.30852e-10,6.60315e-11,1.60104e-08,1.46277e-08,7.0323e-15,2.33684e-08,1.34892e-08,4.85597e-12,3.27434e-11,1.67079e-08,3.06388e-14,1.04576e-11,1.64523e-06,7.84968e-14,1.75388e-12,5.41868e-15,3.34631e-10,6.14631e-13,...,4.89077e-10,2.0974e-08,1.42676e-12,1.45263e-11,1.70107e-12,4.22706e-10,6.84088e-12,1.71566e-10,9.40551e-10,8.15726e-12,5.58772e-10,4.95702e-14,8.384e-13,2.37244e-13,1.56682e-11,2.30056e-05,7.61948e-11,1.63783e-13,1.45474e-09,2.79247e-11,2.80149e-08,6.17159e-15,3.21435e-09,9.93643e-11,1.3224e-13,1.76643e-11,5.50732e-09,3.30071e-11,9.23202e-14,2.53477e-11,6.50509e-13,4.8603e-13,7.1406e-11,8.26829e-14,2.54714e-08,5.09066e-14,8.2217e-12,2.53382e-15,7.48987e-09,5.24636e-14
2,L0DLRE4.jpeg,2.30524e-06,2.3195e-05,5.77912e-11,4.52433e-07,5.0231e-08,1.14304e-10,5.92379e-12,8.12059e-06,1.33027e-07,1.31824e-11,8.94025e-10,0.000266327,2.09235e-09,4.71799e-06,1.17471e-07,1.14772e-10,4.3329e-08,1.44033e-13,4.56283e-12,2.2574e-10,3.51811e-11,8.13545e-09,2.10229e-07,3.16526e-09,1.12275e-05,1.78073e-12,1.36525e-10,4.53442e-14,3.75922e-09,3.1012e-05,1.13428e-10,9.11762e-12,3.50866e-09,1.00998e-11,5.34207e-14,4.73929e-16,7.3872e-13,1.74655e-08,2.9536e-06,...,1.21361e-05,4.98474e-11,2.04136e-15,4.87e-07,2.82326e-11,7.11406e-